# Diving into Pinecone

In [1]:
# loading the API keys in environment variables
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
import pinecone

# initializing pinecone
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),
    environment=os.environ.get('PINECONE_ENV')
)

C:\Users\sthor\anaconda3\envs\NLP\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
pinecone.info.version()

VersionResponse(server='2.0.11', client='2.2.2')

### Pinecone Indexes

In [4]:
pinecone.list_indexes()

[]

#### Creating an index

In [9]:
# creating an index
index_name = 'langchain-pinecone'
if index_name not in pinecone.list_indexes():
    print(f'Creating index {index_name} ....')
    pinecone.create_index(index_name, dimension=1536, metric='cosine', pods=1, pod_type='p1.x2')
    print('Done')
else:
    print(f'Index {index_name} already exists!')

Creating index langchain-pinecone ....
Done


In [6]:
pinecone.describe_index(index_name)

IndexDescription(name='langchain-pinecone', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

#### Getting index statistics

In [11]:
# getting statistics
index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

#### Deleting an index

In [7]:
# deleting an index
index_name = input('Enter Pinecone index to delete: ')
if index_name in pinecone.list_indexes():
    print(f'Deleting index {index_name} ... ')
    pinecone.delete_index(index_name)
    print('Done')
else:
    print(f'Index {index_name} does not exist!')

Enter Pinecone index to delete:  langchain-pinecone


Deleting index langchain-pinecone ... 
Done


#### Inserting into an index

In [12]:
# inserting some random vectors into a Pinecone index
import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]

# vectors id
ids = list('abcde')

In [14]:
ids

['a', 'b', 'c', 'd', 'e']

In [15]:
index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

#### Updating a vector

In [16]:
# updating a vector
index.upsert(vectors=[('c', [0.3] * 1536)])

{'upserted_count': 1}

#### Fetching a vector

In [17]:
# fetching a vector
index = pinecone.Index('langchain-pinecone')
index.fetch(ids=['c', 'd'])

{'namespace': '',
 'vectors': {'c': {'id': 'c',
                   'values': [0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
       

#### Deleting vectors

In [18]:
# deleting vectors
index.delete(ids=['b', 'c'])

{}

In [19]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}

In [20]:
index.fetch(ids=['b'])

{'namespace': '', 'vectors': {}}

In [21]:
index.delete(delete_all=True)

{}

In [24]:
# Volvemos a insertar los vectores
vectors = [[random.random() for _ in range(1536)] for v in range(5)]

# vectors id
ids = list('abcde')

index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

#### Querying

In [25]:
# querying 
queries = [[random.random() for _ in range(1536)] for v in range(2)]

In [26]:
index.query(
    queries=queries,
    top_k=3,
    include_values=False
)

{'results': [{'matches': [{'id': 'b', 'score': 0.749097288, 'values': []},
                          {'id': 'a', 'score': 0.745767772, 'values': []},
                          {'id': 'e', 'score': 0.744436562, 'values': []}],
              'namespace': ''},
             {'matches': [{'id': 'e', 'score': 0.750887632, 'values': []},
                          {'id': 'b', 'score': 0.742472887, 'values': []},
                          {'id': 'a', 'score': 0.742244363, 'values': []}],
              'namespace': ''}]}